<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_15_Traders_trade_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 14 and intro

Script Notes: 

In lesson 14 the agents and their neighbors calculated their marignal rates of substitution and if their MRS were not equal they would then set a price based on the gemometric mean of their rate of subsitution. Now that we have the price we can conduct the trade.   

*Review each class and model set up*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import math
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 33.7 MB/s 
     |████████████████████████████████| 66 kB 4.6 MB/s 


In [3]:
def get_distance(pos_1, pos_2):
    """Calculate Euclidean distance between two positions.
    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    
    used in trader.move()
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

In [4]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [5]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add Trade 

**Part 1**

------------

Now that we have the mrs and the price the next part is to see if it is in the agents best interest to trade. 

To do this, there are two criteria for the agents, (1) will they both be better off and (2) the marginal rates of substitution cannot cross each other. For example if the agent trades so much sugar that they now would prefer to get sugar relative to spice. 

As fair warning trading is fairly involved so we will have layered functions. 


To start we need to calculate the current welfare of each agent so they can know if the trade is making them better off. Luckily we already have a function for this calculate welfare, so we are going to add a note in this function that I am using it in trade (again this is just my personal preference and then I am going to call this function for each agent in the trade function. To help keep my code clean I am going to add it under the calculate mrs and and will use a similar naming convention.

**Part 2 - Build the test exchange**

-------------------------------

Now I will add a function, if the mrs is greater than 1 the agent wants to sell spice, I will call this function may sell spice. and place it above the trade function. Next we have to calculate how much spice and sugar is being traded based on the price. I recommend looking in the book pages 104 through 108 whch explain why we are making these choices, the key takeaway is basing your model on solid theory matters. The amount is fairly simple to determine but we will put it in a seperate function. If the price is greater than 1 then p units of spice are exchanged for one unit of sugar and it if it less than 1 than 1 unit of spice is exchange for 1/p units of sugar.

**Part 3 - Verfiy the exchange is worth it**

-------------------------------

It is important to point out here each exchange is incremental we are only doing a unit of spice or sugar at time and then we will do it again and again until the crtieria of being worse off if an exchange is met. Each increment is a tes to see if it is a good idea. Once we calculate the exchange we need see how that amount of exchange impacts the agents, which is why the function is called maybe sell. Once we calculate the exchange we need to make sure each agent has the resources to conduct the exchange, as an additional verification check.    

Then we need to test the criteria from the book which is are both agents better off and is their welfare crossing. 

Next we make sure the agents are better off, we can just output a true or false statement using calculate welfare function

Then we have to see if their MRS cross. To do this we need to first calculate their MRS, which we have a function for and so we can do another ture or false output. Now this is a little tricky to understand, but based on our trade function, who's ever MRS is greater determines who's maybe sell spice object we call. so we just need to ensure the sell spice of self is greater than other. The key point is which agent is self and which agent is other depends on who's MRS is greater. 

Finally if either criteria is False then we stop the trade cycle. We will also add a print statement just to make sure it is working the way we think. 

This in concludes lesson 15 in lessons 16 we will actually conduct the sale. 


In [ ]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 
      self.prices = []

  
    def get_sugar(self, pos): #part 2
      '''
      used in:
         get_sugar_amount()
         eat()

      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Sugar:
              return agent
      return None
    
    
    def get_sugar_amount(self, pos):
      '''
      used in move()
      '''
      
      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
          return sugar_patch.amount
      return 0
    
    
    def get_spice(self, pos):
      '''
      used in get_spice_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Spice:
              return agent
      return None
        
    
    def get_spice_amount(self, pos):
        '''
        used in move()
        '''

        spice_patch = self.get_spice(pos)
        if spice_patch: 
          return spice_patch.amount
        return 0
  
    def get_trader(self, pos):
      '''
      used in trade_with neighbors()
      '''
      this_cell =self.model.grid.get_cell_list_contents(pos)

      for agent in this_cell: 
        if isinstance(agent, Trader):
          return agent
    
    
    def is_occupied(self, pos): 
      '''
      Helper function for move()
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the example later with none, but right now it wouldn't make any sense
      '''
      Helper function for:
      move()
      trade()

      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
      
    def is_starved(self): 
      '''
      helper function for maybe_die()
      '''

      return (self.sugar <= 0) or (self.spice <=0)

    def calculate_MRS(self): 
      '''
      used in trade()
      '''

      return (self.spice/self.metabolism_spice) / (self.sugar/self.metabolism_sugar)
    
    def calculate_sell_spice_amount(self, price):
      '''
      Helper function for maybe_sell_spice() which is called from trade()
      '''
      if price >= 1:
          sugar = 1
          spice = int(price)
      else:
          sugar = int(1 / price)
          spice = 1
      return sugar, spice
  
    
    
    def maybe_sell_spice(self, other, price, self_welfare, other_welfare):
      '''
      helper function for trade()
      '''
      sugar_exchanged, spice_exchanged = self.calculate_sell_spice_amount(price)

      # Preparing the new sugar spice amount -- what if the exchange were to occur.
      self_sugar = self.sugar + sugar_exchanged
      other_sugar = other.sugar - sugar_exchanged
      self_spice = self.spice - spice_exchanged
      other_spice = other.spice + spice_exchanged

      #Part 3
      #Double Check to see if agents have resources

      if (
          (self_sugar <=0)
       or (other_sugar <= 0)
       or (self_spice <= 0)
       or (other_spice) <= 0)
      ):
       return False

      #trade crtieria 1 are both agents betterr off?
      both_agents_better_off = (
          welfare_self < self.calculate_welfare(self_sugar, self_spice) and 
          (welfare_other < other.calculate_welfare(other_sugar, other_spice)))
      
      #trade criteria 2 is their mrs crossing
      mrs_not_crossing = self.calculate_MRS() > other.calculate_MRS()

      print(both_agents_better_off, mrs_not_crossing)

      if not (both_agents_better_off and mrs_not_crossing):
            return False
       

    
    def trade(self, other):
      '''
      used in trade_with_neighbors()
      '''
      #Part 0
      #sanity check to verify code is working the way we think
      assert self.sugar > 0
      assert self.spice > 0
      assert other.sugar > 0 
      assert other.spice > 0 

      mrs_self = self.calculate_MRS()
      mrs_other = other.calculate_MRS()
      #part 1: 
      welfare_self = self.calculate_welfare(self.sugar, self.spice)
      welfare_other = other.calculate_welfare(other.sugar, other.spice)

      if math.isclose(mrs_self, mrs_other, rel_tol=1e-2): 
        return 

      price = math.sqrt(mrs_self*mrs_other)

      #Part 1
      if mrs_self > mrs_other: 
        sold = self.maybe_sell_spice(other, price, welfare_self, welfare_other)


    ###########################################################################
    #                                                                         #
    #                            MAIN FUNCTIONS                               #
    #                                                                         #
    ###########################################################################

    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which produce the maximum welfare. 

      welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos),
            )
            for pos in neighbors] #part 2

      #find the highest welfare in the cell 
      max_welfare = max(welfares)
      #get the index of maximal welfare cells
      candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare,
                                             rel_tol=1e-02)]
      #convert index to positions of those cells
      candidates = [neighbors[i] for i in candidate_indices]
     
      
      # 3. Find the nearest patch among the candidate.
      min_dist = min([get_distance(self.pos, pos) for pos in candidates])
      final_candidates = [
          pos
          for pos in candidates
          if math.isclose(get_distance(self.pos, pos),min_dist, rel_tol=1e-02)
      ]
      self.random.shuffle(final_candidates)

      # 4. Move agent.
      self.model.grid.move_agent(self, final_candidates[0])



    def eat(self):
      #get sugar
      sugar_patch = self.get_sugar(self.pos)

      if sugar_patch:
        self.sugar = self.sugar - self.metabolism_sugar + sugar_patch.amount
        sugar_patch.amount = 0
        #print(self.sugar, sugar_patch.amount)
      
      #get_spice part 2
      spice_patch = self.get_spice(self.pos)
      #print(spice_patch)
      if spice_patch: 
        self.spice = self.spice - self.metabolism_spice + spice_patch.amount
        spice_patch.amount = 0
        #print(self.spice, spice_patch.amount)

    def maybe_die(self): 

      if self.is_starved(): 
        self.model.grid.remove_agent(self) 
        self.model.schedule.remove(self)
          
    def trade_with_neighbors(self):
      
      neighbor_agents =[self.get_trader(pos) for pos in self.model.grid.get_neighborhood(
          self.pos, self.moore, False, radius=self.vision) if self.is_occupied(pos)]
        
      if len(neighbor_agents) == 0:
            return []
      
      for a in neighbor_agents: 
        if a: 
          self.trade(a)


In [ ]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  
  #part 2
  def randomize_traders(self): 
    """
    Helper function for step()

    updates list of agents used in step function
    """
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)

    return Trader_shuffle
  
  
  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
        
    #part t
    Trader_shuffle = self.randomize_traders()
    
    for agent in Trader_shuffle: 
      agent.move() 
      agent.eat() 
      agent.maybe_die() 

    '''
    #demo error part 1
    for agent in Trader_shuffle:
      agent.move()
    '''
    #part 2
    Trader_shuffle = self.randomize_traders()
    

    #part 2
    for agent in Trader_shuffle: 
      #agent.trade_with_neighbors()
      agent.prices = agent.trade_with_neighbors()
    

    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=1) 



2 1
1 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 2
1 1
1 1
2 1
1 1
1 1
1 1
1 1
1 1
1 1
2 1
2 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 3
1 2
1 2
2 1
1 1
1 1
1 1
1 1
1 1
1 1
2 1
